In [1]:
%pip install langchain langchain_text_splitters langchain_community langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pymupdf

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 70.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.4/764.4 MB 56.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 62.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 56.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 66.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 62.5 MB/s eta 0:00:0000

In [7]:
%pip install faiss-cpu

  Using cached faiss_cpu-1.10.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (4.4 kB)
Using cached faiss_cpu-1.10.0-cp312-cp312-manylinux_2_28_x86_64.whl (30.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


In [9]:
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Single PDF

In [10]:
## 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("./docs/2024000249_과천_디에트르_입주자모집공고문.pdf")
docs = loader.load()

In [11]:
## 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

In [12]:
embeddings = OpenAIEmbeddings(
   api_key=OPENAI_API_KEY,
   model="text-embedding-3-small"
)

In [13]:
## 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)


In [14]:
## 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()


In [15]:
## 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)


In [16]:
## 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


In [17]:
## 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "디에트르는 민간분양이야 공공분양이야?"
response = chain.invoke(question)
print(response)


디에트르는 민간분양입니다.


In [19]:
question = "분양 주택형은 몇개고 전용면적이 각각 어떻게 돼?"
response = chain.invoke(question)
print(response)

분양 주택형은 총 1개이며, 전용면적은 59.9955㎡입니다.


In [20]:
question = "기혼에 자녀가 1명 있는데 다자녀가구 특별공급에 지원할 수 있을까?"
response = chain.invoke(question)
print(response)

아니요, 다자녀가구 특별공급에 지원할 수 없습니다. 다자녀가구 특별공급의 조건 중 하나는 만 19세 미만의 자녀가 2명 이상 있어야 하기 때문입니다. 따라서 자녀가 1명인 경우에는 지원 자격이 없습니다.


In [21]:
question = "고층 분양세대의 분양가는 얼마야?"
response = chain.invoke(question)
print(response)

고층 분양세대의 분양가는 다음과 같습니다:

- 8층~10층: 848,350,000 원
- 11층~15층: 857,150,000 원
- 16층~20층: 865,950,000 원
- 21층~28층: 870,350,000 원

각 층별로 분양가가 다르니 참고하시기 바랍니다.


In [22]:
question = "21층 세대에 당첨이 됐다면, 취등록세와 중도금 대출 이자, 법무사 비용 등등 해서 대략 필요 금액이 얼마나 될까?"
response = chain.invoke(question)
print(response)

죄송하지만, 해당 정보에 대한 구체적인 금액은 제공되지 않았습니다. 따라서 대략적인 필요 금액을 알려드릴 수 없습니다.


In [23]:
question = "너가 계산해줄 수 없어??"
response = chain.invoke(question)
print(response)

모르겠습니다.


In [24]:
question = "과천 어룰림 푸르지오 라비엔오는 공공청약이야?"
response = chain.invoke(question)
print(response)

모르는 정보입니다.


# Multiple PDFs

In [38]:
from langchain_community.document_loaders import FileSystemBlobLoader
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import PyMuPDFParser

loader = GenericLoader(
    blob_loader=FileSystemBlobLoader(
        path="./docs/",
        glob="*.pdf",
    ),
    blob_parser=PyMuPDFParser(),
)
docs = loader.load()

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict



In [50]:
text_splitter2 = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents2 = text_splitter2.split_documents(docs)

In [51]:
embeddings2 = OpenAIEmbeddings(
   api_key=OPENAI_API_KEY,
   model="text-embedding-3-large"
)

In [52]:
## 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore2 = FAISS.from_documents(documents=split_documents2, embedding=embeddings2)

In [53]:
## 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever2 = vectorstore2.as_retriever()

In [54]:
## 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt2 = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [55]:
## 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm2 = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [56]:
## 단계 8: 체인(Chain) 생성
chain2 = (
    {"context": retriever2, "question": RunnablePassthrough()}
    | prompt2
    | llm2
    | StrOutputParser()
)
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question2 = "디에트르는 민간분양이야 공공분양이야?"
response2 = chain2.invoke(question2)
print(response2)


그것에 대한 정보는 제공된 문서에서 찾을 수 없습니다.


In [57]:
question2 = "벨라르테는 민간분양이야 공공분양이야?"
response2 = chain2.invoke(question2)
print(response2)

그에 대한 정보는 제공된 문서에서 찾을 수 없습니다.


In [58]:
question2 = "너가 알고 있는 입주자모집 공고는 몇 개야?"
response2 = chain2.invoke(question2)
print(response2)

모르겠습니다.


In [59]:
question2 = "너가 아는 내용이 뭐야?"
response2 = chain2.invoke(question2)
print(response2)

내가 알고 있는 내용은 과천지식정보타운 린 파밀리에의 입주자 모집 공고문에 대한 정보입니다. 이 공고문에는 주택형 안내, 주변 환경, 교육시설 관련 사항, 단지 내 명칭 및 동표기 변경 가능성, 조경 및 외벽 마감 변경 가능성, 설계 변경 가능성 등이 포함되어 있습니다. 또한, 청약 및 계약 시 주의사항과 관련 법령에 따른 우선 적용 사항 등이 명시되어 있습니다.


In [60]:
question2 = "린 파밀리에는 공공분양이야 민간분양이야?"
response2 = chain2.invoke(question2)
print(response2)

그것에 대한 정보는 제공되지 않았습니다.
